In [49]:
import obspy, os
import pandas as pd
import numpy as np
from obspy.geodetics.base import gps2dist_azimuth

In [2]:
WD_new_lat_lon = pd.read_csv('WD_new.csv',names=['lon','lat'])
WD_bty = pd.read_csv('wd_ChaoJing_new.csv',header=None)
GPStime = pd.read_csv('gpstime.csv',names=['time'])
GPSlat = pd.read_csv('gps_lat.csv',names=['AUV','Buoy','Ship'])
GPSlon = pd.read_csv('gps_lon.csv',names=['AUV','Buoy','Ship'])

In [3]:
itime = 1 
AUV_lat = 25+GPSlat['AUV'].values[itime]/60
AUV_lon = 121+GPSlon['AUV'].values[itime]/60
Ship_lat = 25+GPSlat['Ship'].values[itime]/60
Ship_lon = 121+GPSlon['Ship'].values[itime]/60


In [4]:
## find the bty depth of the ship
minlon = min(WD_new_lat_lon['lon'], key=lambda x:abs(x-Ship_lon))
minlon_index = WD_new_lat_lon['lon'][WD_new_lat_lon['lon']==minlon].index[0]
minlat = min(WD_new_lat_lon['lat'], key=lambda x:abs(x-Ship_lat))
minlat_index = WD_new_lat_lon['lat'][WD_new_lat_lon['lat']==minlat].index[0]
Shipbty = WD_bty[minlon_index][minlat_index]
print(Shipbty)
## find the depth of the AUV
minlon = min(WD_new_lat_lon['lon'], key=lambda x:abs(x-AUV_lon))
minlon_index = WD_new_lat_lon['lon'][WD_new_lat_lon['lon']==minlon].index[0]
minlat = min(WD_new_lat_lon['lat'], key=lambda x:abs(x-AUV_lat))
minlat_index = WD_new_lat_lon['lat'][WD_new_lat_lon['lat']==minlat].index[0]
AUVbty = WD_bty[minlon_index][minlat_index]
print(AUVbty)


-33.694988
-3.0578604


In [73]:
BtyLine = []
SourceDist=[]
for i,lon in enumerate(WD_new_lat_lon['lon']):
    for j,lat in enumerate(WD_new_lat_lon['lat']):
        if AUV_lon<=lon<=Ship_lon and AUV_lat<=lat<=Ship_lat:
        #    print(lon, lat, bty)
            bty = WD_bty[i][j]
            BtyLine.append(bty)
            dist,az,baz = gps2dist_azimuth(Ship_lat,Ship_lon,lat,lon)
            SourceDist.append(dist/1000)

Dist_array = np.array(SourceDist)
Bty_array = np.array(BtyLine)
inds = Dist_array.argsort()
sorted_Bty = Bty_array[inds]
sorted_Dis = sorted(Dist_array)
print(sorted_Bty)
print(sorted_Dis)
       
        

[-32.599056   -31.951574   -32.361691   -31.346395   -31.018929
 -31.646872   -30.941696   -31.583984   -30.723587   -31.351101
 -31.370037   -30.500107   -30.29615    -30.252357   -30.197901
 -30.394701   -30.424181   -29.926519   -30.201963   -29.654074
 -29.625986   -29.57832    -30.247976   -30.299189   -29.627447
 -29.16173    -30.232271   -29.268053   -29.229706   -29.255756
 -29.397989   -28.977951   -28.924006   -30.595022   -28.926918
 -29.156813   -30.336058   -29.071007   -29.061796   -28.908735
 -28.60498    -28.60454    -28.835529   -28.714853   -29.267012
 -28.71702    -29.711943   -28.174931   -28.33646    -28.245924
 -29.384672   -28.535303   -28.035149   -28.335728   -28.461288
 -29.387339   -28.222208   -28.067514   -27.88069    -28.797493
 -27.957096   -27.767855   -27.801306   -28.182697   -27.650429
 -27.591095   -27.571953   -27.802856   -27.46571    -28.068016
 -28.119759   -28.809805   -27.687216   -28.165989   -27.394728
 -27.685705   -27.539686   -27.142424   

In [82]:
totallen = len(sorted_Bty)+1
##Create bty file 
cmd = ''' echo $'\"C\"' > test.bty''' % locals()
os.system(cmd) 
cmd = '''echo %(totallen)s  >> test.bty''' % locals()
os.system(cmd) 
Shipbty = -Shipbty
cmd = ''' echo 0 %(Shipbty)f >> test.bty''' % locals()
os.system(cmd)
for i in range(totallen-1):
    dist = sorted_Dis[i]
    bty  = -1*sorted_Bty[i]
    cmd  = ''' echo %(dist)f %(bty)f >> test.bty ''' %locals()
    os.system(cmd)

In [98]:
envfil = 'AUV-Ship'
titleenv = f'ChaoJing {envfil}'
freq = 18500
Nmedia = 1 
topopt = 'QVW'
option2='A*'
cpbottom = 1600
csbottom = 0.00
den_bottom = 1.800
alpha_bottom = 1.200
NSD = 1 ##number of sourece depth
SD = -Shipbty ##source depth
NRD = 1 ##number of receiver depth
RD = -AUVbty ##receiver depth
NRR  = 1 ## the number of receiver ranges
RR = 1.958000 ##receiver range
runtype = 'A' ##E for eigenray A for amp. delay file(ascii)
Nbeams = 50
alpha = 5
step = 0.000
zbox = 50.000
rbox = max(sorted_Dis)

In [99]:
file = f'{envfil}.env'
f = open(file, 'w')
f.write(f"' {titleenv} ' \n")
f.write(f'{freq} \n')
f.write(f'{Nmedia} \n')
f.write(f'{topopt} \n')
f.write(f'0 0 {-1*min(BtyLine)} \n')
f.write(f'    0.0  1533  /\n')
f.write(f'    {-1*min(BtyLine)}  1533  /\n')
f.write(f' {option2}  0.0 \n')
f.write(f'{-1*min(BtyLine)} {cpbottom} {csbottom} {den_bottom} {alpha_bottom} \n')
f.write(f'! lower halfspace \n')
f.write(f'{NSD}			! NSD \n')
f.write(f'{SD}  ! SD(1:NSD)  ... \n')
f.write(f'{NRD} /		! NRD \n')
f.write(f' {RD}   /		! RD(1:NRD)  ... \n')
f.write(f'{NRR} /		! NRR \n')
f.write(f'{RR}   /		! RR(1)  ... \n')
f.write(f"'{runtype}' \n")
f.write(f'{Nbeams} \n')
f.write(f'{-alpha} {alpha} / \n')
f.write(f'{step} {zbox} {rbox} \n')
f.close()

